In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import torch
from matplotlib import pyplot as plt
from scipy.signal import convolve2d
from torch import tensor, Tensor
import torchvision
from tqdm import tqdm
import os

In [2]:
def generate(x, y):
    x_c = x.clone()
    x_c[:, :10] *= 0.0
    x_c[range(x.shape[0]), y] = x.max()
    return x_c

In [3]:
class Net(torch.nn.Module):
    def __init__(self, dims):
        super().__init__()
        self.layers = []
        self.linear_classifier = LinearClassifier(sum(dims)-784, 10)
        self.linear_classifier.to("cuda")

        for d in range(len(dims) - 1):
            self.layers += [Layer(dims[d], dims[d + 1]).cuda()]

    def predict(self, x):
        goodness_per_label = []
        for label in range(10):
            h = generate(x, label)
            goodness = []
            for layer in self.layers:
                h = layer(h)
                goodness += [h.pow(2).mean(1)]
            goodness_per_label += [sum(goodness).unsqueeze(1)]
        goodness_per_label = torch.cat(goodness_per_label, 1)
        return goodness_per_label.argmax(1)


    def train(self, x_pos, x_neg):
        h_pos, h_neg = x_pos, x_neg
        for i, layer in enumerate(self.layers):
            print('training layer', i, '...')
            h_pos, h_neg = layer.train(h_pos, h_neg)
    def train_last_layer(self,x,y):
        outputs = []
        for layer in self.layers:
            x = layer(x)
            outputs.append(x)
        concatenated_output = torch.cat(outputs, dim=1)
        self.linear_classifier.train_weights(concatenated_output, y)
    def predictu(self,x):
      outputs = []
      for layer in self.layers:
            x = layer(x)
            outputs.append(x)
      concatenated_output = torch.cat(outputs, dim=1)
      return self.linear_classifier.predict(concatenated_output)



In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class LinearClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearClassifier, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

    def train_weights(self, x, y, num_epochs=100):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)


        for epoch in range(num_epochs):
            optimizer.zero_grad()
            outputs = self(x)
            loss = criterion(outputs, y)

            loss.backward(retain_graph=True)
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    def predict(self, x):
        with torch.no_grad():
            outputs = self(x)
            probabilities = nn.functional.softmax(outputs, dim=1)
            _, predicted_class = torch.max(probabilities, 1)
        return predicted_class

In [5]:
import torch
import torch.nn as nn
from torch.optim import Adam

class Layer(nn.Linear):
    def __init__(self, in_features, out_features,
                 bias=True, device=None, dtype=None):
        super().__init__(in_features, out_features, bias, device, dtype)
        self.relu = torch.nn.ReLU()

        self.opt = Adam(self.parameters(), lr=0.02)
        self.threshold = 2.0
        self.num_epochs = 1000


    def forward(self, x):
        x_direction = x / (x.norm(2, 1, keepdim=True) + 1e-4)
        return self.relu(torch.mm(x_direction, self.weight.T) + self.bias.unsqueeze(0))

    def train(self, x_pos, x_neg):
        for i in range(self.num_epochs):
            g_pos = self.forward(x_pos).pow(2).mean(1)
            g_neg = self.forward(x_neg).pow(2).mean(1)
            loss = torch.log(1 + torch.exp(torch.cat([self.threshold - g_pos, g_neg - self.threshold]))).mean()
            self.opt.zero_grad()
            loss.backward()
            self.opt.step()
        return self.forward(x_pos).detach(), self.forward(x_neg).detach()






#Unsupervised

In [6]:
def create_mask(shape, iterations: int = 10):

    blur_filter_1 = np.array(((0, 0, 0), (0.25, 0.5, 0.25), (0, 0, 0)))
    blur_filter_2 = blur_filter_1.T

    image = np.random.randint(0, 2, size=shape)

    for i in range(iterations):
        image = np.abs(convolve2d(image, blur_filter_1, mode='same') / blur_filter_1.sum())
        image = np.abs(convolve2d(image, blur_filter_2, mode='same') / blur_filter_2.sum())

    mask = np.round(image).astype(np.uint8)

    return tensor(mask).to("cuda:0")


def create_negative_image(image_1: Tensor, image_2: Tensor):

    mask = create_mask((image_1.shape[0], image_1.shape[1]))

    image_1 = torch.mul(image_1, mask)
    image_2 = torch.mul(image_2, 1 - mask)

    return torch.add(image_1, image_2)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



torch.manual_seed(1234)
transform = Compose([
        ToTensor(),
        Normalize((0.1300,), (0.3000,)),
        Lambda(lambda x: torch.flatten(x))])

train_loader = DataLoader(MNIST('./data/', train=True,download=True,transform=transform),batch_size=50000, shuffle=False)

test_loader = DataLoader(
    MNIST('./data/', train=False,
          download=True,
          transform=transform),
    batch_size=10000, shuffle=False)
shuffled_train_loader = DataLoader(MNIST('./data/', train=True,download=True,transform=transform),batch_size=50000, shuffle=True)

z, q = next(iter(shuffled_train_loader))
z, q = z.cuda(), q.cuda()
x, y = next(iter(train_loader))
x, y = x.cuda(), y.cuda()
x_neg2=create_negative_image(x, z)

x_neg2 = x_neg2.cuda()

100%|██████████| 9912422/9912422 [00:00<00:00, 93189622.21it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 28952125.67it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28784910.56it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 2264415.64it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [8]:

netu = Net([784, 2000,2000])
netu = netu.to(device)
netu.train(x, x_neg2)

training layer 0 ...
training layer 1 ...


In [9]:
netu.train_last_layer(x,y)

print('train error:', 1.0 - netu.predictu(x).eq(y).float().mean().item())
train_accuracy = netu.predictu(x).eq(y).float().mean().item()
print('train accuracy:', train_accuracy)

x_te, y_te = next(iter(test_loader))
x_te, y_te = x_te.cuda(), y_te.cuda()

print('test error:', 1.0 - netu.predictu(x_te).eq(y_te).float().mean().item())
test_accuracy = netu.predictu(x_te).eq(y_te).float().mean().item()
print('test accuracy:', test_accuracy)

Epoch [10/100], Loss: 0.5712
Epoch [20/100], Loss: 0.3775
Epoch [30/100], Loss: 0.3274
Epoch [40/100], Loss: 0.2798
Epoch [50/100], Loss: 0.2530
Epoch [60/100], Loss: 0.2339
Epoch [70/100], Loss: 0.2207
Epoch [80/100], Loss: 0.2104
Epoch [90/100], Loss: 0.2022
Epoch [100/100], Loss: 0.1954
train error: 0.06192004680633545
train accuracy: 0.9380799531936646
test error: 0.0667000412940979
test accuracy: 0.9332999587059021


به همانند قسمت قبل تابع لاس را تعریف می کنیم.

و وقتی از لیبل ها استفاده نمیکنیم اما همچنان داده غلط و درست داریم و هر بچ داده درست و غلط زا با هم می دهیم تا مذل اموزش داده شود.

در واقع در اینجا یک مدلی آموزش داده می شود بطوریکه اگر ورودی به آن بدهیم خروجی می شود خروجی های هر لایه که با هم کانکت شدند.
این می شود خروجی یک شبکه که با لیبل ها ارتباط خوبی دارد و عکس های تا حدی شبیه به هم خروجی هایی تا حدی شبیه به هم دارند و تا حدی نورون های مشابه ای بیشتر فعال می شوند.

حال که می دانیم تا حدی بردار های خروجی به ازای عکس های ورودی از یک کلاس شبیه به هم می باشند کافیست یک کلاسیفایر اموزش بدهیم تا تشخیص بدهید چه بردار های در جه زمینه هایی مربوط به چه کلاس هایی هستند


و همانطور که در کد دیده می شود در نهایت یک کلاس لینیر که یک لایه هست و در نهایت بردار ای می دهد که از سافت مکس رد می شود اموزش داده می شود.




در نهایت هر داده دیگر اول از شبکه
FF
رد می شود تا وکتور خروجی آن ساخته شود
بعد از آن به لایه کلاسیفایر اموزش داده شده داده می شود و یک لیبل خروجی داده می شود.